In [ ]:
import jiwer
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#model_dir = "wav2vec2-base-960h" # baseline
#model_dir = "wav2vec2-base"
#model_dir = "wav2vec2-base-100h"

#model_dir = "wav2vec2-large-960h"
model_dir = "wav2vec2-large-robust-ft-libri-960h"


In [ ]:
base_directory = Path.cwd().parent

dataset_name = "yale_econ251"
data_dir = base_directory / 'data'

dataset_size = "normal" # or 'tiny'

if dataset_size == "tiny":
    audio_dir = data_dir / 'inputs' / dataset_name / 'lectures-tiny'
    transcripts_dir = data_dir / 'inputs' / dataset_name / 'transcripts-tiny'
else:
    audio_dir = data_dir / 'inputs' / dataset_name / 'lectures'
    transcripts_dir = data_dir / 'inputs' / dataset_name / 'transcripts'

In [ ]:
predictions_dir = data_dir / 'predictions' / dataset_name / model_dir


In [ ]:
transformation = jiwer.Compose([
    jiwer.ToUpperCase(),
    jiwer.RemoveWhiteSpace(replace_by_space=True),
    jiwer.RemoveMultipleSpaces(),
    jiwer.RemovePunctuation(),
    jiwer.ReduceToListOfListOfWords(word_delimiter=" ")
])

In [ ]:
errors = []

# selected_files = ['01', '02', '03', '04', '05' ]
chunk_length = "_cl10"

for transcript in transcripts_dir.glob('*.txt'):
    file_no = transcript.stem
    # if file_no not in selected_files:
    #     continue

    # load the ground truth text
    with open(transcript, 'r') as f:
        ground_truth = f.read()

    # load the predicted text
    pred_file_name = 'pred__' + file_no + "_" + dataset_size
    pred_file_path = (predictions_dir/ dataset_size / pred_file_name).with_suffix('.txt')
    print(pred_file_path)
    with open(pred_file_path, 'r') as f:
        hypothesis = f.read()

    measures = jiwer.compute_measures(ground_truth,
                                      hypothesis,
                                      truth_transform=transformation,
                                      hypothesis_transform=transformation)
    wer = measures['wer']
    mer = measures['mer']
    wil = measures['wil']

    errors.append([wer, mer, wil])


In [ ]:
xrange = [i for i in range(len(errors))]

plt.plot(xrange, np.array(errors)[:,0], label="WER")
plt.plot(xrange, np.array(errors)[:,1], label="MER")
plt.plot(xrange, np.array(errors)[:,2], label="WIL")
plt.title(f"{dataset_name}")
plt.xlabel("lecture no.")
plt.ylabel("Score")
plt.legend()
plt.grid()
plt.show()

print(f"Average WER: {np.average(np.array(errors)[:,0])}")
print(f"Average MER: {np.average(np.array(errors)[:,1])}")
print(f"Average WIL: {np.average(np.array(errors)[:,2])}")


In [ ]:
# Compare chunk lengths

ground_truth_file = transcripts_directory / "01.txt"
with open(ground_truth_file, 'r') as f:
    ground_truth = f.read()


predicted_texts = ["01_cl5.txt", "01_cl10.txt", "01_cl15.txt", "01_cl20.txt"]
errors = []
#
for pred in predicted_texts:
    file_path = predictions_directory / pred

    with open(file_path, 'r') as f:
        hypothesis = f.read()

    error = jiwer.wer(
        ground_truth,
        hypothesis,
        truth_transform=transformation,
        hypothesis_transform=transformation
    )

    measures = jiwer.compute_measures(ground_truth,
                                      hypothesis,
                                      truth_transform=transformation,
                                      hypothesis_transform=transformation)
    wer = measures['wer']
    mer = measures['mer']
    wil = measures['wil']

    errors.append([wer, mer, wil])
    # errors.append(error)

time_taken = [262, 268, 298, 377 ] # in seconds

print(f"WER: {np.array(errors)[:,0]}")
print(f"MER: {np.array(errors)[:,1]}")
print(f"WIL: {np.array(errors)[:,2]}")

print(time_taken)

plt.plot(predicted_texts, errors)
plt.title(f"Compare chunk lengths for lecture 01")
plt.xlabel("Chunk length ")
plt.ylabel("WER")
plt.grid()
plt.show()